In [1]:
#################################################
##### 0-1. Checking the working environment #####
#################################################

def mylib () :
    import os as os
    mylib = input('Write the location of the data file. : ')
    os.chdir(mylib)
    print('Working Directory is now ', os.getcwd())

###########################################
##### 0-2. importing dataset function #####
###########################################    
    
def read() :
    import pandas as pd
    name = input("Enter the data file name (with extension name) : ")
    fm = input("Select the data coding format(1='a b c' or 2='a,b,c'): " )
    if fm == '1':
        form = " "
    elif fm == '2':
        form = ","
    return pd.read_csv(name, sep=form, header=None)
    print('Successfully read the data file')

In [2]:
##################################
##### 1. regressoin function #####
##################################

def regression():
    # import packages
    import numpy as np
    import numpy.linalg as lin
    import pandas as pd
    
    # prompt user to enter the data information
    data = read()
    
    # import data
    response = data[num]
    explanatory = data.drop(num, axis=1)
    
    # design matrix
    n = data.shape[0]
    p = data.shape[1]-1
    one = pd.DataFrame(np.ones((n,1)))
    I = np.eye(n)
    X = pd.concat([pd.DataFrame(one),explanatory], axis=1)
    Y = response
    H = X.dot(lin.inv(X.T.dot(X))).dot(X.T)
    H0 = one.dot(lin.inv(one.T.dot(one))).dot(one.T)
    inv = lin.inv(X.T.dot(X))
    b = inv.dot(X.T).dot(Y).round(3)

    # multiple regression result
    yhat = H.dot(Y).round(1)
    SST = Y.T.dot(I-H0).dot(Y)
    SSE = Y.T.dot(I-H).dot(Y)
    Rsquare = round(1 - SSE/SST, 4)
    MSE = round(SSE/(n-p), 4)

    # output file name
    outputname = input("Write the output file name you want to save (without extension name) : ")
    outputname = outputname+'.txt'

    # outport the result
    with open(outputname,"w") as text_file:
        
        print("Coefficients", file=text_file)
        print("-------------", file=text_file)
        for i in range(p+1):
            if i==0:
                print("Constant:", b[i],sep="   ", file=text_file)
            else: 
                print("Beta",i,":   ",b[i],sep="", file=text_file)
        print("",file=text_file)

        print("ID, Actual values, Fitted values", file=text_file)
        print("--------------------------------", file=text_file)
        for i in range(n):
            print(i+1, Y[i], yhat[i], sep=", ", file=text_file)
        print("",file=text_file)
        
        print("Model Summary", file=text_file)
        print("-------------", file=text_file)
        print("R-square = ", Rsquare, sep="", file=text_file)
        print("MSE = ", MSE, sep="", file=text_file)

In [3]:
#############################
##### 2. Classification #####
#############################

def classification() :
    # import packages
    import os as os
    import math
    import pandas as pd
    import numpy as np
    from numpy.linalg import inv
    from fractions import Fraction as frac
    from pandas import DataFrame as df

    ###################### General background #######################  
    
    # read data
    print('Importing TRAINING dataset')
    train = read()
    print('Importing TEST dataset')
    test = read()
    num = int(input("Enter which column the response variable is recorded: "))-1

    yclass = train[num].unique()
    k = len(yclass) # Assume that values of the class variable are integers starting with 1
    
    # method choice
    choice = int(input("Enter 1 for LDA, 2 for QDA, 3 for RDA , 4 for Logistic Regression, 5 for Naive Bayes or 6 for 1-level decision tree. : "))
    #while ((choice==4&k!=2) | (choice==5$k!=2)) :
    #    choice = int(input("Enter 1 for LDA, 2 for QDA, 3 for RDA , 4 for Logistic Regression or 5 for Naive Bayes. : "))
    #    if ((choice != 4)|(choice!=5)) :
    #        break   
    
    
    if choice <= 3 :
        print(' It has',k,'Classes.','\n', 
              'Input each priors with ascending order of Class index.','\n',
              'ex) Priors of 3 class size = 1/3, 1/3, 1/3','\n')
        prior = input('Priors : ').split(',')
        prior = list(map(lambda x: frac(x), prior))
        if not len(prior) == k :
            prior = [1/k]*k
            print("Your prior input does not correscond with class size.")
            print("So equal prior is given such as", prior)
        elif not sum(prior)==1 :
            prior = [1/k]*k
            print("Sum of your prior input is not equal to 1.")
            print("So equal prior is given such as", prior)

        # Basic matrixs
        n = train.shape[0]
        nk = train[num].groupby(train[num]).count()
        n_t = test.shape[0]
        x = train.drop(num, axis=1).T
        x_t = test.drop(num, axis=1).T
        y = train[num]
        y_t = test[num]
        c = []
        c_t = []
        p = train.shape[1]-1
        means = train.groupby(train[num]).mean().T
        sk = train.groupby(train[num]).cov()
        sp = 0

        for i in range(1, k+1):
            sp = sp + (nk[i]-1)*sk.loc[i]/(n-k)

    ###############
    ### (i) LDA ###
    ###############
    
    if choice == 1 :
        # Training data
        d = (means.T).dot(inv(sp)).dot(x) + np.array(np.repeat(np.diag((-0.5)*means.T.dot(inv(sp)).dot(means)), n)).reshape(4,n) + np.repeat(np.log(prior),n).reshape(4,n) 
        c = df(d).idxmax()

         # Test data
        d_t = (means.T).dot(inv(sp)).dot(x_t) + np.array(np.repeat(np.diag((-0.5)*means.T.dot(inv(sp)).dot(means)), n_t)).reshape(4,n_t) + np.repeat(np.log(prior),n_t).reshape(4,n_t) 
        c_t = df(d_t).idxmax()

    ################
    ### (ii) QDA ###
    ################
    
    if choice == 2 :
        # (ii) QDA
        # Training data
        d = np.zeros((k,n))
        for i in range(1, k+1):
            d[i-1] = np.repeat((-0.5)*np.log(lin.det(sk.loc[i])),n) + np.diag((-0.5)*(x.apply(lambda x : x-means[i])).T.dot(inv(sk.loc[i])).dot(x.apply(lambda x : x-means[1]))) + np.repeat(np.log(prior[i-1]),n) 
        c = df(d, index=yclass).idxmax()

        # Test data
        d_t = np.zeros((k,n_t))
        for i in range(1, k+1):
            d_t[i-1] = np.repeat((-0.5)*np.log(lin.det(sk.loc[i])),n_t) + np.diag((-0.5)*(x_t.apply(lambda x : x-means[i])).T.dot(inv(sk.loc[i])).dot(x_t.apply(lambda x : x-means[1]))) + np.repeat(np.log(prior[i-1]),n_t) 
        c_t = df(d_t, index=yclass).idxmax()

    #################
    ### (iii) RDA ###
    #################
        
    if choice == 3 :
        #Alpha & gamma selection by 0.05
        alpha = np.arange(0, 1.05, 0.05)
        gamma = np.arange(0, 1.05, 0.05)
        d_temp = np.zeros((k, n))
        s_rda = np.zeros((p,p))
        sigma = np.diag(sp).mean()
        result = []

        for a in alpha :
            for g in gamma :
                for i in yclass :
                    s_rda = a*sk.loc[i] + (1-a)*(g*sp+(1-g)*sigma*np.eye(p))
                    d_temp[i-1] = np.repeat((-0.5)*np.log(lin.det(s_rda)),n) + np.diag((-0.5)*(x.apply(lambda x : x-means[i])).T.dot(inv(s_rda)).dot(x.apply(lambda x : x-means[i]))) + np.repeat(np.log(prior[i-1]),n) 
                c_temp = df(d_temp, index=yclass).idxmax()
                accuracy_temp = sum(c_temp==y)/n
                accuracy_rate = [a,g, accuracy_temp]
                result.append(accuracy_rate)

            result = df(result, index=range(1, len(result)+1), columns=['alpha', 'gamma', 'accuracy rate'])

        # Accuracy plot
        import matplotlib.pyplot as plt
        from mpl_toolkits.mplot3d import Axes3D
        xss, yss = np.meshgrid(alpha,gamma)
        z = np.array(result['accuracy rate'])
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(xss, yss, z, s=10, c='r')
        ax.set_xlabel('Alpha')
        ax.set_ylabel('Gamma')
        ax.set_zlabel('Accuracy rate')
        plt.show()

        # Optimal parameters
        optimal_loc = result['accuracy rate'].idxmax()
        optimal = df(result.loc[optimal_loc])

        print('Optimal parameters are given as follows.', '\n')
        print(optimal)
        a = int(optimal.loc['alpha'])
        g = int(optimal.loc['gamma'])

        # Training data
        d = np.zeros((k,n))
        for i in yclass :
            s_rda_op = a*sk.loc[i] + (1-a)*(g*sp + (1-g)*sigma*np.eye(p))
            d[i-1] =  np.repeat((-0.5)*np.log(lin.det(s_rda_op)),n) + np.diag((-0.5)*(x.apply(lambda x : x-means[i])).T.dot(lin.inv(s_rda_op)).dot(x.apply(lambda x : x-means[i]))) + np.repeat(np.log(prior[i-1]),n) 
        c = df(d, index=yclass).idxmax()

        # Test data
        d_t = np.zeros((k,n_t))
        for i in range(1, k+1):
            s_rda_op = a*sk.loc[i] + (1-a)*(g*sp+(1-g)*sigma*np.eye(p))
            d_t[i-1] =  np.repeat((-0.5)*np.log(lin.det(s_rda_op)),n_t) + np.diag((-0.5)*(x_t.apply(lambda x : x-means[i])).T.dot(lin.inv(s_rda_op)).dot(x_t.apply(lambda x : x-means[i]))) + np.repeat(np.log(prior[i-1]),n_t) 
        c_t = df(d_t, index=yclass).idxmax()

    #######################################
    ### Output setting for (i)(ii)(iii) ###
    #######################################
    
    if choice <= 3 :

        print('Please enter the number of the rows you can have.','\n')
        out_num = int(input('Enter the number : '))

        # Crosstable
        con = df({'Actual Class':y, 'Predicted Class':c})
        table = pd.crosstab(con['Actual Class'],con['Predicted Class'], colnames=[''])
        accuracy = np.trace(table)/n

        con_t = df({'Actual Class':y_t, 'Predicted Class':c_t})
        table_t = pd.crosstab(con_t['Actual Class'],con_t['Predicted Class'], colnames=[''])
        accuracy_t = np.trace(table_t)/n_t

        # output file
        outputname = input("Write the output file name you want to save (without extension name) : ")
        outputname = outputname+'.txt'

        with open(outputname,"w") as text_file:

            print('ID, Actual class, Resub pred', file=text_file)
            print('-----------------------------', file=text_file)
            for i in range(0, out_num):
                 print(i+1, y[i], c[i], sep=', ', file=text_file)
            print('(continue)',file=text_file)        
            print('',file=text_file)
            print('Confusion Matrix (Resubstitution)', file=text_file)
            print('----------------------------------','\n','             Predicted Class', file=text_file)
            print(table, file=text_file)
            print("",file=text_file)
            print("Model Summary (Resubstitution)", file=text_file)
            print('------------------------------', file=text_file)
            print("Overall accuracy = ", accuracy.round(3), sep='', file=text_file)
            print('', file=text_file)

            print('ID, Actual class, Test pred', file=text_file)
            print('-----------------------------', file=text_file)
            for i in range(0, out_num):
                print(i+1, y_t[i], c_t[i], sep=', ', file=text_file)
            print('(continue)',file=text_file)        
            print('',file=text_file)
            print('Confusion Matrix (Test)', file=text_file)
            print('----------------------------------','\n','             Predicted Class', file=text_file)
            print(table_t, file=text_file)
            print("",file=text_file)
            print("Model Summary (Test)", file=text_file)
            print('------------------------------', file=text_file)
            print("Overall accuracy = ", accuracy_t.round(3), sep='', file=text_file)
            print('', file=text_file)
        print("Output file has been successfully saved in ",os.getcwd(),"/",outputname,sep="") 

    
    ################################
    ### (iv) Logistic Regression ###
    ################################
    
    if choice == 4 :
        # import packages
        import math
        import pandas as pd
        import numpy as np
        from pandas import DataFrame as df
        from scipy.optimize import minimize

        # Basic matrix for Logistic
        n = train.shape[0]
        n_t = test.shape[0]
        ex = train.drop(num, axis=1)
        x = pd.concat([pd.DataFrame(np.repeat(1,n)),ex], axis=1)
        ex_t = test.drop(num, axis=1)
        x_t = pd.concat([pd.DataFrame(np.repeat(1,n_t)),ex_t], axis=1)
        y = train[num]
        y_t = test[num]
        p = train.shape[1]

        def loglike(B):
            return - np.dot((y-1).T,x).dot(B) + sum(np.log(1+np.exp(np.dot(x,B))))
        loglik = minimize(loglike, np.repeat(0,p))
        b = loglik.x
        print('The result that maximizes Log-Likelihood of Logistic Regression(MLE method).')
        print(df(b))

        cutoff = input('Enter the cutoff value from 0 to 1. : ')
        cutoff = float(cutoff)
        if not (0<=cutoff<=1) : 
            cutoff=0.5
            print('The cutoff value must be in 0 to 1. So set 0.5 as the cutoff value.')

        # Training data
        prob = np.round(np.exp(np.dot(x,b))/(1+np.exp(np.dot(x,b))), 3)
        c = np.repeat(0,n)
        c[prob >= cutoff]=2
        c[prob <= cutoff]=1

        # Test data
        prob_t = np.round(np.exp(np.dot(x_t,b))/(1+np.exp(np.dot(x_t,b))), 3)
        c_t = np.repeat(0,n_t)
        c_t[prob_t >= cutoff]=2
        c_t[prob_t <= cutoff]=1

        # Output setting
        out_num = int(input('Please enter the maximum output row you want to have in the output file. :' ))

        # Crosstable
        con = df({'Actual Class':y, 'Predicted Class':c})
        table = pd.crosstab(con['Actual Class'],con['Predicted Class'], colnames=[''])
        accuracy = np.trace(table)/n
        sensi = table.loc[2,2]/sum(table.loc[2,])
        speci = table.loc[1,1]/sum(table.loc[1,])

        con_t = df({'Actual Class':y_t, 'Predicted Class':c_t})
        table_t = pd.crosstab(con_t['Actual Class'],con_t['Predicted Class'], colnames=[''])
        accuracy_t = np.trace(table_t)/n_t
        sensi_t = table_t.loc[2,2]/sum(table_t.loc[2,])
        speci_t = table_t.loc[1,1]/sum(table_t.loc[1,])

        # output file
        outputname = input("Write the output file name you want to save (without extension name) : ")
        outputname = outputname+'.txt'

        with open(outputname,"w") as text_file:

            print('ID, Actual class, Resub pred, Pred Prob', file=text_file)
            print('-----------------------------', file=text_file)
            for i in range(0, out_num):
                 print(i+1, y[i], c[i], prob[i], sep=', ', file=text_file)
            print('(continue)',file=text_file)        
            print('',file=text_file)

            print('Confusion Matrix (Resubstitution)', file=text_file)
            print('----------------------------------','\n','             Predicted Class', file=text_file)
            print(table, file=text_file)
            print("",file=text_file)

            print("Model Summary (Resubstitution)", file=text_file)
            print('------------------------------', file=text_file)
            print("Overall accuracy = ", accuracy.round(3), sep='', file=text_file)
            print("Sensitivity = ", sensi.round(3), sep='', file=text_file)
            print("Specificity = ", speci.round(3), sep='', file=text_file)
            print('', file=text_file)

            print('ID, Actual class, Test pred, Pred Prob', file=text_file)
            print('-----------------------------', file=text_file)
            for i in range(0, out_num):
                print(i+1, y_t[i], c_t[i], prob[i], sep=', ', file=text_file)
            print('(continue)',file=text_file)        
            print('',file=text_file)

            print('Confusion Matrix (Test)', file=text_file)
            print('----------------------------------','\n','             Predicted Class', file=text_file)
            print(table_t, file=text_file)
            print("",file=text_file)

            print("Model Summary (Test)", file=text_file)
            print('------------------------------', file=text_file)
            print("Overall accuracy = ", accuracy_t.round(3), sep='', file=text_file)
            print("Sensitivity = ", sensi_t.round(3), sep='', file=text_file)
            print("Specificity = ", speci_t.round(3), sep='', file=text_file)
            print('', file=text_file)
        print("Output file has been successfully saved in ",os.getcwd(),"/",outputname,sep="") 
        

    ###########################
    ##### (v) Naive Bayes #####
    ###########################

    if choice == 5 :

        import pandas as pd
        from pandas import DataFrame as df
        import numpy as np
        from scipy.stats import norm

        # Basic matrixs
        n = train.shape[0]
        n_t = test.shape[0]
        x = train.drop(num, axis=1)
        x_t = test.drop(num, axis=1)
        y = train[num]
        y_t = test[num]
        p = train.shape[1]

        # set variable attributes
        print(' It has',k,'variables.','\n', 
              'Input variable number which has numerical attribute.','\n',
              'ex) 1, 3, 5','\n')
        numer = input('Numerical attributes : ').split(',')
        numer = (np.array(numer).astype('int')-1).tolist()

        # probabiliy of training set
        t1 = train[train.loc[:,num]==1]
        t2 = train[train.loc[:,num]==2]
        prob_1 = np.ones((n,p))
        prob_2 = np.ones((n,p))

        nnn=list(range(0,p))
        nnn.remove(num)

        for i in range(0,n) :
            for j in nnn :
                if(train.loc[i,j] != '?') : 
                    if(j in numer) :
                        prob_1[i,j] = norm.pdf(train.loc[i,j], loc=np.mean(t1.loc[:,j]), scale=np.std(t1.loc[:,j]) )
                        prob_2[i,j] = norm.pdf(train.loc[i,j], loc=np.mean(t2.loc[:,j]), scale=np.std(t2.loc[:,j]) )
                    else :
                        t = pd.crosstab(y, x.loc[:,j]).apply(lambda r: r/r.sum(), axis=1)
                        prob_1[i,j] = t.loc[1,train.loc[i,j]]
                        prob_2[i,j] = t.loc[2,train.loc[i,j]]

        prob_temp = np.c_[np.prod(prob_1, axis=1), np.prod(prob_2, axis=1) ]
        prob = np.zeros((n,2))
        prob[:,0] = np.round(prob_temp[:,0]/prob_temp.sum(axis=1),3)
        prob[:,1] = np.round(prob_temp[:,1]/prob_temp.sum(axis=1),3)
        c = df(prob, columns=[1,2]).idxmax(axis=1)

        # probability of test set
        prob_1_t = np.ones((n_t,p))
        prob_2_t = np.ones((n_t,p))

        for i in range(0,n_t) :
            for j in nnn :
                if(test.loc[i,j] != '?') : 
                    if(j in numer) :
                        prob_1_t[i,j] = norm.pdf(test.loc[i,j], loc=np.mean(t1.loc[:,j]), scale=np.std(t1.loc[:,j]) )
                        prob_2_t[i,j] = norm.pdf(test.loc[i,j], loc=np.mean(t2.loc[:,j]), scale=np.std(t2.loc[:,j]) )
                    else :
                        t = pd.crosstab(y, x.loc[:,j]).apply(lambda r: r/r.sum(), axis=1)
                        prob_1_t[i,j] = t.loc[1,test.loc[i,j]]
                        prob_2_t[i,j] = t.loc[2,test.loc[i,j]]

        prob_temp_t = np.c_[np.prod(prob_1_t, axis=1), np.prod(prob_2_t, axis=1) ]
        prob_t = np.zeros((n_t,2))
        prob_t[:,0] = np.round(prob_temp_t[:,0]/prob_temp_t.sum(axis=1),3)
        prob_t[:,1] = np.round(prob_temp_t[:,1]/prob_temp_t.sum(axis=1),3)
        prob_t

        c_t = df(prob_t, columns=[1,2]).idxmax(axis=1)

        # Output setting
        out_num = int(input('Please enter the maximum output row you want to have in the output file. :' ))

        # Crosstable
        con = df({'Actual Class':y, 'Predicted Class':c})
        table = pd.crosstab(con['Actual Class'],con['Predicted Class'], colnames=[''])
        accuracy = np.trace(table)/n
        sensi = table.loc[2,2]/sum(table.loc[2,])
        speci = table.loc[1,1]/sum(table.loc[1,])

        con_t = df({'Actual Class':y_t, 'Predicted Class':c_t})
        table_t = pd.crosstab(con_t['Actual Class'],con_t['Predicted Class'], colnames=[''])
        accuracy_t = np.trace(table_t)/n_t
        sensi_t = table_t.loc[2,2]/sum(table_t.loc[2,])
        speci_t = table_t.loc[1,1]/sum(table_t.loc[1,])

        # output file
        outputname = input("Write the output file name you want to save (without extension name) : ")
        outputname = outputname+'.txt'
        import os as os
        with open(outputname,"w") as text_file:

            print('ID, Actual class, Resub pred, Pred Prob', file=text_file)
            print('-----------------------------', file=text_file)
            for i in range(0, out_num):
                 print(i+1, y[i], c[i], prob[i][c[i]-1], sep=', ', file=text_file)
            print('(continue)',file=text_file)        
            print('',file=text_file)

            print('Confusion Matrix (Resubstitution)', file=text_file)
            print('----------------------------------','\n','             Predicted Class', file=text_file)
            print(table, file=text_file)
            print("",file=text_file)

            print("Model Summary (Resubstitution)", file=text_file)
            print('------------------------------', file=text_file)
            print("Overall accuracy = ", accuracy.round(3), sep='', file=text_file)
            print("Sensitivity = ", sensi.round(3), sep='', file=text_file)
            print("Specificity = ", speci.round(3), sep='', file=text_file)
            print('', file=text_file)

            print('ID, Actual class, Test pred, Pred Prob', file=text_file)
            print('-----------------------------', file=text_file)
            for i in range(0, out_num):
                print(i+1, y_t[i], c_t[i], prob[i][c[i]-1], sep=', ', file=text_file)
            print('(continue)',file=text_file)        
            print('',file=text_file)

            print('Confusion Matrix (Test)', file=text_file)
            print('----------------------------------','\n','             Predicted Class', file=text_file)
            print(table_t, file=text_file)
            print("",file=text_file)

            print("Model Summary (Test)", file=text_file)
            print('------------------------------', file=text_file)
            print("Overall accuracy = ", accuracy_t.round(3), sep='', file=text_file)
            print("Sensitivity = ", sensi_t.round(3), sep='', file=text_file)
            print("Specificity = ", speci_t.round(3), sep='', file=text_file)
            print('', file=text_file)

        print("Output file has been successfully saved in ",os.getcwd(),"/",outputname,sep="")
        
    #####################################
    ##### (v) 1-level decision tree #####
    #####################################
    
    if choice == 6 :
        
        # Basic matrixs
        n = train.shape[0]
        n_t = test.shape[0]
        x = train.drop(num, axis=1)
        x_t = test.drop(num, axis=1)
        y = train[num]
        y_t = test[num]
        p = train.shape[1]
        
        nnn=list(range(0,p))
        nnn.remove(num)
        classes = np.unique(y)
        classes.sort
        
        # Greedy search for continuous variable
        GoS = []
        split = []
        for j in nnn :
            value = np.unique(x[j])
            value.sort()
            t = len(value)
            goodness = []
            s = []
            print('Start Greedy Search for', nnn[j]+1, 'th column.')
            print('Repeat ',t-1,'times.','\n')

            for i in range(0,(t-1)) :
                s.append(np.mean(value[[i, (i+1)]]))
                #for i in range(0, (t-1)) :
                print('Split at ', s[i])
                t1 = train.loc[ train[j] <= s[i], num ]
                t2 = train.loc[ train[j] > s[i], num ]
                n = len(y)
                n1 = len(t1)
                n2 = len(t2)
                # Gini impurity
                imp_t1 = 1- (sum(t1==classes[0])/n1)**2 - (sum(t1==classes[1])/n1)**2
                imp_t2 = 1- (sum(t2==classes[0])/n2)**2 - (sum(t2==classes[1])/n2)**2
                # Goodness of split
                imp_t = 1- (sum(y==classes[0])/n)**2 - (sum(y==classes[1])/n)**2
                goodness.append(imp_t - n1/n*imp_t1 - n2/n*imp_t2)
                print('Goodness of split is ', np.round(goodness[i],4))

            GoS.append(max(goodness))
            split.append(s[goodness.index(max(goodness))])
            print('\n')

        # result
        result = np.c_[split, GoS]
        result = pd.DataFrame(result, columns=['split', 'Goodness of Split'], index=nnn)
        print(result)
        j = result.idxmax()['Goodness of Split']
        var_1 = result.index[j]
        split_1 = result['split'][j]
        print('Node 1 is ', var_1, 'and split at ', split_1)
        
        # training set
        t1 = train.loc[ train[j] <= split_1, num ]
        t2 = train.loc[ train[j] > split_1, num ]
        c = np.repeat(0,n)
        c[t1.index] = classes[0]
        c[t2.index] = classes[1]

        # test set
        t1_t = test.loc[ test[j] <= split_1, num ]
        t2_t = test.loc[ test[j] > split_1, num ]
        c_t = np.repeat(0,n_t)
        c_t[t1_t.index] = classes[0]
        c_t[t2_t.index] = classes[1]

        # Crosstable
        con = df({'Actual Class':y, 'Predicted Class':c})
        table = pd.crosstab(con['Actual Class'],con['Predicted Class'], colnames=[''])
        accuracy = np.trace(table)/n
        sensi = table.iloc[1,1]/sum(table.iloc[1])
        speci = table.iloc[0,0]/sum(table.iloc[0])

        con_t = df({'Actual Class':y_t, 'Predicted Class':c_t})
        table_t = pd.crosstab(con_t['Actual Class'],con_t['Predicted Class'], colnames=[''])
        accuracy_t = np.trace(table_t)/n_t
        sensi_t = table_t.iloc[1,1]/sum(table_t.iloc[1])
        speci_t = table_t.iloc[0,0]/sum(table_t.iloc[0])
        
        # output file
        out_num = int(input('Please enter the maximum output row you want to have in the output file. :' ))
        outputname = input("Write the output file name you want to save (without extension name) : ")
        outputname = outputname+'.txt'
        with open(outputname,"w") as text_file:
            print('Tree Structure', file=text_file)
            print('\t', 'Node 1: ', var_1, ' <= ', split_1, ' (', y.tolist().count(classes[0]),',', y.tolist().count(classes[1]), ')', file=text_file)
            print('\t', 'Node 2: ', classes[0], ' (', t1.tolist().count(classes[0]),',', t1.tolist().count(classes[1]), ')', file=text_file)
            print('\t', 'Node 3: ', classes[0], ' (', t2.tolist().count(classes[0]),',', t2.tolist().count(classes[1]), ')', file=text_file)

            print('ID, Actual class, Resub pred', file=text_file)
            print('-----------------------------', file=text_file)
            for i in range(0, out_num):
                 print(i+1, y[i], c[i], sep=', ', file=text_file)
            print('(continue)',file=text_file)        
            print('',file=text_file)

            print('Confusion Matrix (Resubstitution)', file=text_file)
            print('----------------------------------','\n','             Predicted Class', file=text_file)
            print(table, file=text_file)
            print("",file=text_file)

            print("Model Summary (Resubstitution)", file=text_file)
            print('------------------------------', file=text_file)
            print("Overall accuracy = ", accuracy.round(3), sep='', file=text_file)
            print("Sensitivity = ", sensi.round(3), sep='', file=text_file)
            print("Specificity = ", speci.round(3), sep='', file=text_file)
            print('', file=text_file)

            print('ID, Actual class, Test pred', file=text_file)
            print('-----------------------------', file=text_file)
            for i in range(0, out_num):
                print(i+1, y_t[i], c_t[i], sep=', ', file=text_file)
            print('(continue)',file=text_file)        
            print('',file=text_file)

            print('Confusion Matrix (Test)', file=text_file)
            print('----------------------------------','\n','             Predicted Class', file=text_file)
            print(table_t, file=text_file)
            print("",file=text_file)

            print("Model Summary (Test)", file=text_file)
            print('------------------------------', file=text_file)
            print("Overall accuracy = ", accuracy_t.round(3), sep='', file=text_file)
            print("Sensitivity = ", sensi_t.round(3), sep='', file=text_file)
            print("Specificity = ", speci_t.round(3), sep='', file=text_file)
            print('', file=text_file)

        print("Output file has been successfully saved in ",os.getcwd(),"/",outputname,sep="")

In [4]:
def HW7() :
    print('Checking the working environment.')
    mylib()
    ans = int(input('Enter 1 to use Regression, Enter 2 to use Classification : '))
    if ans == 1 : 
        regression_analysis()
    elif ans == 2 :
        classification()

In [5]:
HW7() #C:/Users/User/Desktop/18-2/DM/data

Checking the working environment.
Write the location of the data file. : C:/Users/User/Desktop/18-2/DM/data
Working Directory is now  C:\Users\User\Desktop\18-2\DM\data
Enter 1 to use Regression, Enter 2 to use Classification : 2
Importing TRAINING dataset
Enter the data file name (with extension name) : harris.dat
Select the data coding format(1='a b c' or 2='a,b,c'): 1
Importing TEST dataset
Enter the data file name (with extension name) : harris.dat
Select the data coding format(1='a b c' or 2='a,b,c'): 1
Enter which column the response variable is recorded: 5
Enter 1 for LDA, 2 for QDA, 3 for RDA , 4 for Logistic Regression, 5 for Naive Bayes or 6 for 1-level decision tree. : 6
Start Greedy Search for 1 th column.
Repeat  25 times. 

Split at  3960.0
Goodness of split is  0.0026
Split at  4155.0
Goodness of split is  0.0052
Split at  4335.0
Goodness of split is  0.0079
Split at  4410.0
Goodness of split is  0.0223
Split at  4470.0
Goodness of split is  0.0254
Split at  4560.0
Goodn